In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D, ReLU, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [16]:
no_of_merged_samples = 5000
no_of_params = 2

In [17]:
#signal_bbh_df = pd.read_csv("C:/Users/sunda/Documents/Github/Gravitational Wave Detection/scripts/raw-data-files/Final_BBH_Merged_Noise_Signal_Reduced_No_ABS.csv", header=None)
#signal_bbh = signal_bbh_df.astype(float)

#noise_df = pd.read_csv("C:/Users/sunda/Documents/Github/Gravitational Wave Detection/scripts/raw-data-files/Final_Merged_Noise_Reduced_No_Abs.csv",header = None)
#noise = noise_df.astype(float)

#val_signal_df = pd.read_csv("C:/Users/sunda/Documents/Github/Gravitational Wave Detection/scripts/raw-val-data-files/val_Final_BBH_Merged_Noise_Signal_Reduced_No_ABS.csv", header = None)
#val_signal_bbh = val_signal_df.astype(float)

#val_noise_df = pd.read_csv("C:/Users/sunda/Documents/Github/Gravitational Wave Detection/scripts/raw-val-data-files/val_Final_Merged_Noise_Reduced_No_Abs.csv", header = None)
#val_noise = val_noise_df.astype(float)


signal_bbh = pd.read_csv("C:/Users/sunda/Documents/Github/Gravitational Wave Detection/scripts/raw-data-files-enhanced-data-generation/Final_BBH_Merged_Noise_Signal_Reduced_No_ABS.csv", header=None).astype(float)
noise = pd.read_csv("C:/Users/sunda/Documents/Github/Gravitational Wave Detection/scripts/raw-data-files-enhanced-data-generation/Final_Merged_Noise_Reduced_No_Abs.csv", header=None).astype(float)
val_signal_bbh = pd.read_csv("C:/Users/sunda/Documents/Github/Gravitational Wave Detection/scripts/raw-val-data-files-enhanced-data-generation/val_Final_BBH_Merged_Noise_Signal_Reduced_No_ABS.csv", header=None).astype(float)
val_noise = pd.read_csv("C:/Users/sunda/Documents/Github/Gravitational Wave Detection/scripts/raw-val-data-files-enhanced-data-generation/val_Final_Merged_Noise_Reduced_No_Abs.csv", header=None).astype(float)


In [18]:
print(signal_bbh.shape)
print(noise.shape)
print(val_signal_bbh.shape)
print(val_noise.shape)

(5000, 16384)
(5000, 16384)
(1000, 16384)
(1000, 16384)


In [19]:
signal_bbh['label'] = 1
noise['label'] = 0
val_signal_bbh['label'] = 1
val_noise['label'] = 0

In [20]:
combined_data = pd.concat([signal_bbh, noise], axis = 0).reset_index(drop = True)
combined_val_data = pd.concat([val_signal_bbh, val_noise], axis = 0).reset_index(drop = True)

In [21]:
shuffled_data = shuffle(combined_data, random_state=42)
shuffled_val_data = shuffle(combined_val_data, random_state=42)

In [8]:
shuffled_data.to_csv("C:/Users/sunda/Documents/Github/Gravitational Wave Detection/scripts/raw-data-files/Final_Shuffled_Data.csv", index=False, header=False)
shuffled_val_data.to_csv("C:/Users/sunda/Documents/Github/Gravitational Wave Detection/scripts/raw-data-files/Final_Shuffled_Val_Data.csv", index=False, header=False)

In [22]:
X = shuffled_data.iloc[:, :-1] 
#X *= 1e19
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = shuffled_data.iloc[:, -1]    

X_v = shuffled_val_data.iloc[:, :-1]
X_v = scaler.transform(X_v)
y_v = shuffled_val_data.iloc[:, -1]

In [23]:
X_train, _, y_train, _ = train_test_split(X, y, test_size=1) #, random_state=1
X_val, _, y_val, _     = train_test_split(X_v , y_v, test_size=1) #, random_state=1

In [24]:
model = Sequential()
model.add(Conv1D(16, 16, input_shape=(16384, 1)))
model.add(MaxPool1D(4, 4))
model.add(ReLU())
model.add(Dropout(0.2))
model.add(Conv1D(32, 8))
model.add(MaxPool1D(4, 4))
model.add(ReLU())
model.add(Dropout(0.2))
model.add(Conv1D(64, 8))
model.add(MaxPool1D(4, 4))
model.add(ReLU())
model.add(Dropout(0.2))
model.add(Conv1D(128, 8))
model.add(MaxPool1D(4, 4))
model.add(ReLU())
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # ✅ Only 1 output neuron

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 16369, 16)         272       
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 4092, 16)          0         
 g1D)                                                            
                                                                 
 re_lu_4 (ReLU)              (None, 4092, 16)          0         
                                                                 
 dropout_3 (Dropout)         (None, 4092, 16)          0         
                                                                 
 conv1d_5 (Conv1D)           (None, 4085, 32)          4128      
                                                                 
 max_pooling1d_5 (MaxPoolin  (None, 1021, 32)          0         
 g1D)                                                 

In [26]:
if X_train.ndim == 2:
    X_train = np.expand_dims(X_train, axis=-1)
if X_val.ndim == 2:
    X_val = np.expand_dims(X_val, axis=-1)

y_train = y_train.astype(np.float32)
y_val = y_val.astype(np.float32)

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    batch_size=16, epochs=20,
                    callbacks=[early_stop])

Epoch 1/20
625/625 [==============================] - 10s 13ms/step - loss: 0.4012 - accuracy: 0.7428 - val_loss: 0.1262 - val_accuracy: 0.9630
Epoch 2/20
625/625 [==============================] - 8s 12ms/step - loss: 0.0796 - accuracy: 0.9715 - val_loss: 0.1150 - val_accuracy: 0.9705
Epoch 3/20
625/625 [==============================] - 8s 13ms/step - loss: 0.0680 - accuracy: 0.9766 - val_loss: 0.1268 - val_accuracy: 0.9625
Epoch 4/20
625/625 [==============================] - 9s 14ms/step - loss: 0.0532 - accuracy: 0.9815 - val_loss: 0.1200 - val_accuracy: 0.9665
Epoch 5/20
625/625 [==============================] - 10s 16ms/step - loss: 0.0532 - accuracy: 0.9825 - val_loss: 0.1305 - val_accuracy: 0.9710


In [27]:
loss, accuracy = model.evaluate(X_val, y_val)
print(f"✅ Validation Accuracy: {accuracy * 100:.2f}%")

63/63 [==============================] - 1s 9ms/step - loss: 0.1150 - accuracy: 0.9705
✅ Validation Accuracy: 97.05%


(9999, 16384)